## imprt libraries

In [59]:
import random
import pandas as pd
import numpy as np

## read datasets and preprocessing

In [60]:
truck_data = pd.read_csv("truck.csv")


order_data = pd.read_csv("order_large.csv")
order_data["Weight"] = order_data["Weight"] / 10000
order_data["Area"] = order_data["Area"] / 10000

distance_data = pd.read_csv("distance.csv")
distance_data["Distance(M)"] = distance_data["Distance(M)"] / 1000
truck_data

,Truck Type,Inner Size,Weight Capacity,Cost Per KM,speed
0,0,16.1x2.5,10000,3,40
1,1,12.1x2.5,5000,2,40
2,2,9.1x2.3,2000,1,40


In [61]:
order_data=order_data.iloc[0:1000,:]

## create geen pool

In [62]:
# fixed geanpool
order_ids = order_data["Order_ID"]
order_ids = pd.DataFrame(order_ids)
order_ids =  order_ids.drop_duplicates()
order_ids

,Order_ID
0,A140109
9,A140110
18,A140112
27,A190223
63,A190225
...,...
872,A240111
908,A240112
944,A240104
966,A080376


In [38]:
# len(order_ids.values)
# order_details=order_data[order_data["Order_ID"]== "A140109"].iloc[0, :]
# order_details

# order_details = order_data[order_data["Order_ID"]== "A140109"].iloc[0, :]
# order_details["Weight"]

# truck_detail = truck_detail(individual[0])
# order_geenpool = order_ids 
# order_geenpool = order_geenpool.drop(index=[0])
# order_ids

In [63]:
def truck_detail(type):
    truck_detail = [truck_data.loc[type, i] for i in truck_data.columns]
    return pd.Series(truck_detail, index=["Truck Type", "Inner Size", "Weight Capacity", "Cost Per KM" , "speed"]) 

## create individual

In [65]:
def init_individuals():
    individuals = []
    order_geenpool = order_ids 
    while len(order_geenpool) > 0:  # while there are orders
        truck_type = random.choice([0, 1, 2])
        truck_details = truck_detail(truck_type)
        inner_size_components = truck_details["Inner Size"].split('x')
        truck_area = float(inner_size_components[0]) * float(inner_size_components[1])
        order_list = []
        sum_weight = 0
        sum_area = 0
        while len(order_geenpool) > 0:
            random_order_idx = random.randint(0, len(order_geenpool) - 1)
            order_id = order_geenpool.iloc[random_order_idx, 0]
            order_details = order_data[order_data["Order_ID"] == order_id].iloc[0]
            # first constraint 
            if (order_details["Weight"] + sum_weight) <= truck_details["Weight Capacity"] and \
               (order_details["Area"] + sum_area) <= truck_area:
                order_list.append(order_id)
                sum_weight += order_details["Weight"]
                sum_area += order_details["Area"]
                order_geenpool = order_geenpool[order_geenpool['Order_ID'] != order_id]
            else:
                break
        if len(order_list) == 0:
            pass 
        else:
            individuals.append((truck_type, order_list))
    return individuals

# individuals = init_individuals()
# print(individuals)

## initialize population

In [66]:
def init_population(size):
    population = []
    for i in range(size) :
        idi = init_individuals()
        population.append(idi)
    return population

In [67]:
population = init_population(10)
population

[[(0, ['A240425', 'A250557', 'A240111', 'A250402']),
  (1, ['A220306', 'A190223']),
  (1, ['A220492', 'A240104', 'A220312']),
  (0,
   ['A190225',
    'A230155',
    'A260061',
    'A220318',
    'A190226',
    'A250346',
    'A220493']),
  (1, ['A240042', 'A260603']),
  (0, ['A230668', 'A230525', 'A080376', 'A240061', 'A250401']),
  (0,
   ['A240110',
    'A230669',
    'A240108',
    'A220310',
    'A240045',
    'A260604',
    'A270114']),
  (0, ['A260605', 'A220313', 'A240046', 'A230676', 'A260606']),
  (1, ['A220303', 'A250556']),
  (0, ['A260078', 'A250554', 'A230670']),
  (0, ['A240112', 'A220319', 'A230153', 'A240426']),
  (0, ['A250403', 'A250558', 'A220308']),
  (0, ['A140109', 'A230675', 'A240109', 'A220301', 'A220321']),
  (0, ['A220494', 'A140110', 'A080377', 'A240281']),
  (0, ['A140112', 'A250404', 'A240049', 'A220300']),
  (1, ['A250555']),
  (2, ['A240107']),
  (0, ['A230151', 'A260376', 'A220309']),
  (1, ['A260052']),
  (0, ['A230154', 'A240050', 'A230667'])],
 [(1, 

## functionality 

In [68]:
def calculate_distance_route(route):
    total_distance = 0
    source = "City_61"  # Assuming the starting location is the warehouse
    
    # Iterate over each order in the route
    for order_id in route:
        # Get the destination for the current order from the order_data dataset
        destination = order_data.loc[order_data["Order_ID"] == order_id, "Destination"].iloc[0]
        
        # Get the distance between the source and destination from the distance_data dataset
        distance = distance_data[(distance_data["Source"] == source) & (distance_data["Destination"] == destination)]
        
        # Check if the distance is found
        if not distance.empty:
            # Add the distance to the total distance
            total_distance += distance.iloc[0]["Distance(M)"]
            # Update the source for the next order
            source = destination
        elif(source == destination):
            total_distance +=0
        else:
            print(f"Error: Distance not found for route {source} -> {destination}")
            return None  # Return None if distance is not found
    
    # Add the distance from the last destination back to the warehouse
    distance_back = distance_data[(distance_data["Source"] == source) & (distance_data["Destination"] == "City_61")]
    if not distance_back.empty:
        total_distance += distance_back.iloc[0]["Distance(M)"]
    elif(source == "City_61"):
        total_distance +=0
    else:
        print(f"Error: Distance not found for route {source} -> city_61")
        return None  # Return None if distance is not found
    
    return total_distance

# Example usage
route = ['A210204', 'A150402', 'A250599', 'A090422', 'A190598']
distance = calculate_distance_route(population[0][0][1])
print("Total distance:", distance)



Total distance: 8664.715


In [69]:
def costForTruck(truck_type,distance):
    if truck_type == 0 :
        cost = distance * 3
    elif truck_type == 1:
        cost = distance * 2
    else:
        cost = distance
    return cost 
        

In [70]:
costForTruck(0,distance)

25994.145

In [15]:
!pip install datetime

     -------------------------------------- 52.6/52.6 kB 902.3 kB/s eta 0:00:00


In [71]:
from datetime import datetime
speed = 40
stop_time = 2
def calculate_arrival_time(route):
    travel_time_days_per_orders = []
    
    for i in range(len(route)) :
        total_distance = calculate_distance_route(route[0:i+1])
        if i == 0:
            days = (total_distance / speed) / 24
            travel_time_days_per_orders.append(round(days))
        else:
            days = ((total_distance / speed) + 2*i) / 24
            travel_time_days_per_orders.append(round(days)) 
    return travel_time_days_per_orders  
        
# calculate_arrival_time(['A220356', 'A250272'])

def spiltDate(date_string):
    date_components = date_string.split()[0]
    # Split the date components by '/' to extract month, day, and year
    month, day, year = map(int, date_components.split('-'))
    return day

In [72]:
# order_data[order_data["Order_ID"] == "A140109"].iloc[0,5]

# spiltDate(order_data[order_data["Order_ID"] == "A140109"].iloc[0,5])

In [73]:
def fitness_function(individual):
    cost_individual = 0.0
    for truck in individual:
        distance_for_truck = calculate_distance_route(truck[1])
        if distance_for_truck is None:
            distance_for_truck = 0  # Penalize the solution if distance calculation fails
        cost_for_truck = costForTruck(truck[0], distance_for_truck) # Only get the cost, not the time
        cost_individual += cost_for_truck
        if cost_for_truck is not None:
            cost_individual += cost_for_truck
        else:
            cost_individual += 0  # Penalize the solution if cost calculation fails
            
    arrival_time_for_ord =  calculate_arrival_time(truck[1])
    i=0
    for ord_id in truck[1]:
        Available_Time = order_data[order_data["Order_ID"] == ord_id].iloc[0,5]
        Deadline = order_data[order_data["Order_ID"] == ord_id].iloc[0,6]
        Available_day = spiltDate(Available_Time)
        Deadline_day = spiltDate(Deadline)
        arrival_time = arrival_time_for_ord[i] + Available_day
        # this is penlity
        if arrival_time > Deadline_day:
            cost_individual += 10 
    return round(cost_individual)  # Higher fitness for lower cost

# Example usage
fitness_score = fitness_function(population[1])
print("Fitness score:", fitness_score)


Fitness score: 564332


In [74]:
def tournament_selection(population):
    selected_parents = []
    
    for _ in range(2):  
        candidates = random.sample(population, 5)
        candidates_fitness = []
        for sol in candidates:
            candidates_fitness.append(fitness_function(sol))
        
        # Assess the prowess of each candidate, and let the brilliance shine forth
        best_fitness = min(candidates_fitness)  # Select the one with the highest fitness
        idx_best =  candidates_fitness.index(best_fitness)
        chosen_one = candidates[idx_best]
        # Marvel at the chosen one and add it to the list of selected parents
        selected_parents.append(chosen_one)
    
    return selected_parents

# Example usage:
# parents = tournament_selection(population)
# print("Selected parents:", parents)
# print(parents)

In [75]:
import random

def roulette_wheel_selection(population):
    # Calculate the total fitness score of the population
    total_fitness = sum(fitness_function(individual) for individual in population)
    
    # Generate a random number between 0 and the total fitness score
    spin = random.uniform(0, total_fitness)
    
    # Spin the roulette wheel and select the individual based on their cumulative fitness score
    cumulative_fitness = 0
    for individual in population:
        cumulative_fitness += fitness_function(individual)
        if cumulative_fitness >= spin:
            return individual

# Example usage:
# selected_individual = roulette_wheel_selection(population)
# print("Selected individual:", selected_individual)



In [76]:
import random

def one_point_crossover(parent1, parent2):
    # Choose a random crossover point
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    
    # Create the offspring by combining the parents' genes
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return offspring1, offspring2

# offspring1, offspring2 = one_point_crossover(population[0],population[1])

# fp1 = fitness_function(population[0])
# fp2 = fitness_function(population[1])

# fo1 = fitness_function(offspring1)
# fo2 = fitness_function(offspring2)

# print(fp1,"\n",fp2,"\n",fo1,"\n",fo2,"\n")





In [77]:
import random

def uniform_crossover(parent1, parent2, probability=0.5):
    # Create the offspring by selecting genes from parents with a given probability
    offspring1 = []
    offspring2 = []
    
    for gene1, gene2 in zip(parent1, parent2):
        if random.random() < probability:
            offspring1.append(gene1)
            offspring2.append(gene2)
        else:
            offspring1.append(gene2)
            offspring2.append(gene1)
    
    return offspring1, offspring2

# offspring1, offspring2 = uniform_crossover(population[0],population[1])
# fp1 = fitness_function(population[0])
# fp2 = fitness_function(population[1])

# fo1 = fitness_function(offspring1)
# fo2 = fitness_function(offspring2)

# print(fp1,"\n",fp2,"\n",fo1,"\n",fo2,"\n")


In [78]:
def shuffle_mutation(individual):
    for truck in range(len(individual)):
        random.shuffle(individual[truck][1])
    return individual


# print(fitness_function(shuffle_mutation(population[3])))
# print(fitness_function(population[3]))





In [79]:
import random

def mutation(individual):
    # Randomly shuffle a subset of genes
    if len(individual) >= 2:
        subset_size = random.randint(1, min(3, len(individual)))  # Randomly select subset size (up to 3)
        subset_indices = random.sample(range(len(individual)), subset_size)  # Randomly select subset indices
        random.shuffle(subset_indices)  # Shuffle the subset indices
        # Shuffle genes at selected indices
        shuffled_genes = [individual[i] for i in subset_indices]
        random.shuffle(shuffled_genes)
        # Update individual with shuffled subset
        for i, index in enumerate(subset_indices):
            individual[index] = shuffled_genes[i]
    return individual

# mutation(population[0])

In [80]:
def elitism_selection(population, num_elites):
    # Sort the population based on fitness in descending order
    sorted_population = sorted(population, key=lambda ind: ind[1], reverse=True)
    # Select the top individuals (elites) to survive
    elites = sorted_population[:num_elites]
    
    return elites

# Example usage:
# num_elites = 50  # Set the number of elites to survive
# elites = elitism_selection(population, num_elites)
# print("Elite individuals:", len(elites))

In [81]:
def generational_replacement(population, offspring):
    # Combine the current population and the offspring
    combined_population = population + offspring
    
    # Sort the combined population based on fitness in descending order
    sorted_population = sorted(combined_population, key=lambda ind: ind[1], reverse=True)
    
    # Select the top individuals to survive, equal to the size of the original population
    new_population = sorted_population[:len(population)]
    
    return new_population



In [83]:
import random

# Define constants
# POPULATION_SIZE = 25
# NUM_GENERATIONS = 30
# TOURNAMENT_SIZE = 5
# MUTATION_RATE = 0.1
# CROSSOVER_RATE = 0.8

# Genetic algorithm function
def genetic_algorithm(POPULATION_SIZE = 25,NUM_GENERATIONS = 30,TOURNAMENT_SIZE = 5,MUTATION_RATE = 0.1,CROSSOVER_RATE = 0.8):
    # Initialize population
    population = init_population(POPULATION_SIZE)
    
    # Main loop for the number of generations
    for generation in range(NUM_GENERATIONS):
        # Evaluate fitness of each individual in the population
        fitness_scores = [fitness_function(individual) for individual in population]
        
        # Select parents for crossover using tournament selection
        parents = [tournament_selection(population) for _ in range(POPULATION_SIZE)]
        # [[p1,p2],[p1,p2]]
        # Perform crossover to create offspring
        offspring = []
        for i in range(0, POPULATION_SIZE, 2):
            if random.random() < CROSSOVER_RATE:
                child1, child2 = one_point_crossover(parents[i][0], parents[i][1])
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.extend([parents[i][0], parents[i][1]])
        
        # Perform mutation on the offspring
        for i in range(len(offspring)):
            if random.random() < MUTATION_RATE:
                offspring[i] = mutation(offspring[i])
        
        # Replace the current population with the offspring
        population = offspring
        
        # Print the best individual in the current generation
        best_individual = min(zip(population, fitness_scores), key=lambda x: x[1])[0]
        print(f"Generation {generation+1}, Best Individual: {best_individual} \n fitness= {fitness_function(best_individual)}")
        # print(f"Generation {generation+1}, Best Individual: {best_individual}")
    
    # Return the best individual from the final generation
    return min(zip(population, fitness_scores), key=lambda x: x[1])[0]

# Example usage
best_solution = genetic_algorithm()
print("Best solution found:", best_solution)



Generation 1, Best Individual: [(0, ['A250556', 'A220301', 'A240104', 'A230676']), (0, ['A230153', 'A240110', 'A240425', 'A240112', 'A250403']), (1, ['A220300', 'A270114', 'A220312']), (2, ['A220493']), (1, ['A250555', 'A260606']), (0, ['A250346', 'A250554', 'A220492', 'A220313', 'A240107']), (0, ['A190223', 'A140112', 'A190225', 'A240042', 'A220319']), (1, ['A240045', 'A260061']), (2, ['A220318']), (1, ['A260078']), (0, ['A140109', 'A260605', 'A220494', 'A140110']), (0, ['A250403', 'A190223', 'A250346', 'A230669', 'A220309']), (0, ['A240281', 'A220492', 'A250554', 'A250556']), (0, ['A240109', 'A230153', 'A240049', 'A230667', 'A190225']), (1, ['A260604', 'A230670', 'A270114']), (0, ['A230668', 'A190226', 'A260603', 'A240104', 'A220319']), (1, ['A240042']), (1, ['A240425', 'A230154']), (1, ['A240107', 'A230675', 'A240112']), (1, ['A230676', 'A220321']), (0, ['A220312', 'A260052', 'A250555']), (0, ['A220303', 'A240110', 'A230151', 'A250402']), (2, ['A080377']), (0, ['A250404', 'A240061',

In [ ]:
distance_data[(distance_data["Source"] == "City_61")][distance_data["Destination"] == "City_54"].iloc[0,2]
# order_data[order_data["Order_ID"] == "A140109"].iloc[0,4]

In [ ]:
destenion = order_data[order_data["Order_ID"] == "A140109"].iloc[0,4]
destenion
source=distance_data[(distance_data["Source"] == "City_61")] \
                    [distance_data["Destination"] == "City_54"].iloc[0,2]
                    
# source[source["Destination"] == "City_47"]
source

